In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-driver

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-10 06:00:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.01MB/s    in 0.2s    

2022-03-10 06:00:55 (5.01 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in Amazon Data from S3 into a DataFrame
url = ' '
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Musical_Instruments_v1_00.tsv'), sep='\t', header=True)

In [ ]:
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [ ]:

df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [ ]:
df.count()

904765

In [ ]:
df = df.drop('marketplace', 'review_headline', 'review_body', 'product_category', 'verified_purchase')

In [ ]:
df.show(10)

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|          3|            0|          1|   N| 2015-08-31|
|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|          5|            0|          0|   N| 2015-08-31|
|    6111003| RIZR67JKUDBI0|B0006VMBHI|     603261968|AudioQuest LP rec...|          3|            0|          1|   N| 2015-08-31|
|    1546619|R27HL570VNL85F|B002B55TRG|     575084461|Hohner Inc. 560BX...|          5|            0|          0|   N| 2015-08-31|
|   12222213|R34EBU9QDWJ1GD|B00N1YPXW2|     165236328|Blue Yeti USB Mic...|        

In [ ]:
df

DataFrame[customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, review_date: string]

In [ ]:
df = df.withColumn("customer_id", df.customer_id.cast('int'))
df = df.withColumn("product_parent", df.product_parent.cast('int'))
df = df.withColumn("star_rating", df.star_rating.cast('int'))
df = df.withColumn("helpful_votes", df.helpful_votes.cast('int'))
df = df.withColumn("total_votes", df.total_votes.cast('int'))
df = df.withColumn("review_date", df.review_date.cast('date'))

In [ ]:
df

DataFrame[customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, review_date: date]

## Creating dataframes for each table

In [ ]:
review_id_table_df = df[['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']]

In [ ]:
review_id_table_df.show(30)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RMDCHWD0Y5OZ9|   45610553|B00HH62VB6|     618218723| 2015-08-31|
| RZSL0BALIYUNU|   14640079|B003LRN53I|     986692292| 2015-08-31|
| RIZR67JKUDBI0|    6111003|B0006VMBHI|     603261968| 2015-08-31|
|R27HL570VNL85F|    1546619|B002B55TRG|     575084461| 2015-08-31|
|R34EBU9QDWJ1GD|   12222213|B00N1YPXW2|     165236328| 2015-08-31|
|R1WCUI4Z1SIQEO|   46018513|B001N4GRGS|     134151483| 2015-08-31|
| RL5LNO26GAVJ1|   10225065|B009PJRMHQ|     694166585| 2015-08-31|
|R3GYQ5W8JHP8SB|    6356995|B00NKBDAZS|     446431775| 2015-08-31|
|R30SHYQXGG5EYC|   35297198|B006MIU7U2|     125871705| 2015-08-31|
|R14YLXA56NP51I|   32139520|B000FIBD0I|     771888534| 2015-08-31|
|R1ZH0HSH38IOTZ|   36060782|B0002E52GG|      68535945| 2015-08-31|
|R3H53KLLC210XI|    5301309|B00RZIH52G|     725541773| 2015-08

In [ ]:
product_table_df = df[['product_id', 'product_title']]

In [ ]:
product_table_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00HH62VB6|AGPtek® 10 Isolat...|
|B003LRN53I|Sennheiser HD203 ...|
|B0006VMBHI|AudioQuest LP rec...|
|B002B55TRG|Hohner Inc. 560BX...|
|B00N1YPXW2|Blue Yeti USB Mic...|
|B001N4GRGS|Middle Atlantic P...|
|B009PJRMHQ|Kmise 1pc Pickgua...|
|B00NKBDAZS|Kealoha Concert U...|
|B006MIU7U2|Halco 80000 - MR1...|
|B000FIBD0I|Gator GPTBLACK Pl...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
customer_table_df = df['customer_id']

In [ ]:
vine_table_df = df[['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']]

## Cleaning the dataframes

### Review ID Table DF

In [ ]:
review_id_table_df

DataFrame[review_id: string, customer_id: int, product_id: string, product_parent: int, review_date: date]

In [ ]:
review_id_table_df.count()

904765

In [ ]:
review_id_table_df.drop_duplicates(['review_id']).count()

904765

### Product Table DF

In [ ]:
product_table_df #Product id and product title

DataFrame[product_id: string, product_title: string]

In [ ]:
product_table_df.count()

904765

In [ ]:
product_table_df = product_table_df.drop_duplicates(['product_id'])

### Customer Table DF

In [ ]:
customer_table_df #customer id and customer count

Column<'customer_id'>

In [ ]:
customer_table_df = df.groupby('customer_id').count().toDF('customer_id', 'customer_count')

In [ ]:
customer_table_df

DataFrame[customer_id: int, customer_count: bigint]

### Vine Table DF

In [ ]:
vine_table_df

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string]

Postgres Setup

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<databaseendpoint>:5432/<database>"
config = {"user":"<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to review_id table in RDS

review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to product table in RDS

product_table_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to customer table in RDS

customer_table_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write dataframe to vine table in RDS

vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)